In [ ]:
import models
import data
import itertools
import multiprocessing as mp
import os
import numpy as np

In [ ]:
models_todo = filter(lambda m: ("aux" not in m) and not ("vgg" in m and not "bn" in m) and (not "noresidual" in m) and ("lenet" in m), models.all_classifiers.keys())
data_todo = ['cifar10', 'cifar100', 'mnist', 'kmnist', 'fashionmnist', 'cinic10']
gpus = [0, 1, 2, 3]
workers_per_gpu = 2

def gpu_worker(gpu, q):
    while not q.empty():
        (model, dataset) = q.get()
        !python train.py --data_dir /data/datasets/ --output_dir .output/ --classifier $model --dataset $dataset --batch_size 256 --gpu_id $gpu --num_workers 8 --scheduler "Step" --max_epochs 125 --cudnn_non_deterministic

if __name__ == '__main__':
    que = mp.Queue()
    for model_name in models_todo:
        for ds in data_todo:
            que.put((model_name, ds))
    
    processes = []
    for gpu in gpus * workers_per_gpu:
        p = mp.Process(target=gpu_worker, args=(gpu, que))
        p.start()
        processes.append(p)

In [ ]:
for p in processes:
    p.kill()